# Bank Loans Data - Hypothesis Testing

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import re

import scipy.stats as stats

# set the graphs to show in the jupyter notebook
%matplotlib inline

# set seabor graphs to a better style
sns.set(style="ticks")

from dateutil.relativedelta import relativedelta

In [2]:
# Load the data
bankloans = pd.read_csv( 'LoansData.csv' )
bankloans

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,20000.0,20000.00,8.90%,36 months,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,735-739,14.0,14272.0,2.0,< 1 year
1,19200.0,19200.00,12.12%,36 months,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,715-719,12.0,11140.0,1.0,2 years
2,35000.0,35000.00,21.98%,60 months,debt_consolidation,23.81%,CA,MORTGAGE,11500.00,690-694,14.0,21977.0,1.0,2 years
3,10000.0,9975.00,9.99%,36 months,debt_consolidation,14.30%,KS,MORTGAGE,3833.33,695-699,10.0,9346.0,0.0,5 years
4,12000.0,12000.00,11.71%,36 months,credit_card,18.78%,NJ,RENT,3195.00,695-699,11.0,14469.0,0.0,9 years
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,30000.0,29950.00,16.77%,60 months,debt_consolidation,19.23%,NY,MORTGAGE,9250.00,705-709,15.0,45880.0,1.0,8 years
2496,16000.0,16000.00,14.09%,60 months,home_improvement,21.54%,MD,OWN,8903.25,740-744,18.0,18898.0,1.0,10+ years
2497,10000.0,10000.00,13.99%,36 months,debt_consolidation,4.89%,PA,MORTGAGE,2166.67,680-684,4.0,4544.0,0.0,10+ years
2498,6000.0,6000.00,12.42%,36 months,major_purchase,16.66%,NJ,RENT,3500.00,675-679,8.0,7753.0,0.0,5 years


In [3]:
#Cleaning up the column names

bankloans.columns = [i.replace ('.','_') for i in bankloans.columns]

In [4]:
# Data Type conversions:

bankloans['Interest_Rate'] = bankloans['Interest_Rate'].str.replace('%','').astype('float')

bankloans['Debt_To_Income_Ratio'] = bankloans['Debt_To_Income_Ratio'].str.replace('%','').astype('float')

bankloans['Loan_Length'] = bankloans['Loan_Length'].str.replace('months','').astype('int')

bankloans['Employment_Length'] = bankloans['Employment_Length'].str.replace('years','').str.replace('year','').str.replace('<','').str.replace('+','').astype('float')

bankloans['FICO_Range'] = bankloans['FICO_Range'].str.split('-',expand = True)[0].astype('float')

C:\Users\Toshiba\AppData\Local\Temp/ipykernel_8456/1253487411.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  bankloans['Employment_Length'] = bankloans['Employment_Length'].str.replace('years','').str.replace('year','').str.replace('<','').str.replace('+','').astype('float')


In [5]:
bankloans.dtypes

Amount_Requested                  float64
Amount_Funded_By_Investors        float64
Interest_Rate                     float64
Loan_Length                         int32
Loan_Purpose                       object
Debt_To_Income_Ratio              float64
State                              object
Home_Ownership                     object
Monthly_Income                    float64
FICO_Range                        float64
Open_CREDIT_Lines                 float64
Revolving_CREDIT_Balance          float64
Inquiries_in_the_Last_6_Months    float64
Employment_Length                 float64
dtype: object

In [6]:
# Identify the duplicates if any

bankloans.duplicated().sum()/bankloans.duplicated().count() #no duplicate values found

# Missing values detection and treatment

def missing_treatment(x):
    if x.dtype == 'object':
        x = x.fillna(x.mode()[0])
    else:
        x = x.fillna(x.median())
    return x
 
# Outliers Detection and Treatment

def  outlier_treatment(x):
    if ((x.dtype == 'float') or (x.dtype == 'int')):
        x.clip(lower = x.quantile(0.01), upper = x.quantile(0.99))
    else:
        x
    return x

In [7]:
bankloans = bankloans.apply(lambda x:missing_treatment(x))
bankloans = bankloans.apply(lambda x:outlier_treatment(x))

In [8]:
bankloans.isna().sum()

Amount_Requested                  0
Amount_Funded_By_Investors        0
Interest_Rate                     0
Loan_Length                       0
Loan_Purpose                      0
Debt_To_Income_Ratio              0
State                             0
Home_Ownership                    0
Monthly_Income                    0
FICO_Range                        0
Open_CREDIT_Lines                 0
Revolving_CREDIT_Balance          0
Inquiries_in_the_Last_6_Months    0
Employment_Length                 0
dtype: int64

# Interest rate is varied for different loan amounts.

In [9]:
Loan_vs_rate = bankloans[['Amount_Requested','Interest_Rate']]
Loan_vs_rate.head(10).sort_values(by = 'Amount_Requested', ascending = True)

,Amount_Requested,Interest_Rate
5,6000.0,15.31
9,7000.0,6.91
3,10000.0,9.99
6,10000.0,7.90
4,12000.0,11.71
8,14675.0,14.33
1,19200.0,12.12
0,20000.0,8.90
7,33500.0,17.14
2,35000.0,21.98


**Null hypothesis (Ho): r = 0**
    
**Alternate Hypothesis (Ha) : r <> 0**
    
**Confidence Interval : 95% ; p-value : 0.05**

In [10]:
relate_1 = stats.pearsonr(Loan_vs_rate.Amount_Requested, Loan_vs_rate.Interest_Rate)
relate_1

(0.3324401743401225, 1.4231505725298107e-65)

***Correlation coefficient r = 0.332. There is a weak positive relationship between the Loan amounts and the interest rates.***

# Loan length impact on interest rate.

In [11]:
bankloans

,Amount_Requested,Amount_Funded_By_Investors,Interest_Rate,Loan_Length,Loan_Purpose,Debt_To_Income_Ratio,State,Home_Ownership,Monthly_Income,FICO_Range,Open_CREDIT_Lines,Revolving_CREDIT_Balance,Inquiries_in_the_Last_6_Months,Employment_Length
0,20000.0,20000.00,8.90,36,debt_consolidation,14.90,SC,MORTGAGE,6541.67,735.0,14.0,14272.0,2.0,1.0
1,19200.0,19200.00,12.12,36,debt_consolidation,28.36,TX,MORTGAGE,4583.33,715.0,12.0,11140.0,1.0,2.0
2,35000.0,35000.00,21.98,60,debt_consolidation,23.81,CA,MORTGAGE,11500.00,690.0,14.0,21977.0,1.0,2.0
3,10000.0,9975.00,9.99,36,debt_consolidation,14.30,KS,MORTGAGE,3833.33,695.0,10.0,9346.0,0.0,5.0
4,12000.0,12000.00,11.71,36,credit_card,18.78,NJ,RENT,3195.00,695.0,11.0,14469.0,0.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,30000.0,29950.00,16.77,60,debt_consolidation,19.23,NY,MORTGAGE,9250.00,705.0,15.0,45880.0,1.0,8.0
2496,16000.0,16000.00,14.09,60,home_improvement,21.54,MD,OWN,8903.25,740.0,18.0,18898.0,1.0,10.0
2497,10000.0,10000.00,13.99,36,debt_consolidation,4.89,PA,MORTGAGE,2166.67,680.0,4.0,4544.0,0.0,10.0
2498,6000.0,6000.00,12.42,36,major_purchase,16.66,NJ,RENT,3500.00,675.0,8.0,7753.0,0.0,5.0


In [12]:
bankloans.Loan_Length.nunique()

2

In [13]:
Time_vs_rate = bankloans[['Loan_Length','Interest_Rate']]
Time_vs_rate

,Loan_Length,Interest_Rate
0,36,8.90
1,36,12.12
2,60,21.98
3,36,9.99
4,36,11.71
...,...,...
2495,60,16.77
2496,60,14.09
2497,36,13.99
2498,36,12.42


In [14]:
Time_vs_rate.groupby('Loan_Length')['Interest_Rate'].mean()

Loan_Length
36    12.127187
60    16.407464
Name: Interest_Rate, dtype: float64

**Null hypothesis (Ho): There is no relation between loan length and the interest rate.**
    
**Alternate Hypothesis (Ha) : There is a relation between loan length and interest rate.**
    
**Confidence Interval : 95% ; p-value : 0.05**

In [15]:
s1 = Time_vs_rate.loc[bankloans.Loan_Length == 36,'Interest_Rate']
s2 = Time_vs_rate.loc[bankloans.Loan_Length == 60,'Interest_Rate']

In [16]:
relate_2 = stats.f_oneway(s1,s2)
relate_2

F_onewayResult(statistic=548.0367191411436, pvalue=9.605991607837365e-110)

***p-value <<< 0.05, which means Loan length directly impacts the rate of interest.***

# Interest rate varies for different purpose of loans.

In [17]:
bankloans.Loan_Purpose.nunique()

14

In [18]:
bankloans.Loan_Purpose.value_counts()

debt_consolidation    1307
credit_card            444
other                  201
home_improvement       152
major_purchase         101
small_business          87
car                     50
wedding                 39
medical                 30
moving                  29
vacation                21
house                   20
educational             15
renewable_energy         4
Name: Loan_Purpose, dtype: int64

In [19]:
Purpose_vs_rate = bankloans[['Loan_Purpose','Interest_Rate']]
Purpose_vs_rate

,Loan_Purpose,Interest_Rate
0,debt_consolidation,8.90
1,debt_consolidation,12.12
2,debt_consolidation,21.98
3,debt_consolidation,9.99
4,credit_card,11.71
...,...,...
2495,debt_consolidation,16.77
2496,home_improvement,14.09
2497,debt_consolidation,13.99
2498,major_purchase,12.42


In [20]:
Purpose_vs_rates = pd.DataFrame(Purpose_vs_rate.groupby('Loan_Purpose')['Interest_Rate'].mean().round(2))
Purpose_vs_rates = Purpose_vs_rates.reset_index().sort_values(by = 'Interest_Rate', ascending = False)
Purpose_vs_rates.reset_index()

,index,Loan_Purpose,Interest_Rate
0,8,moving,13.62
1,2,debt_consolidation,13.59
2,5,house,13.45
3,9,other,13.16
4,1,credit_card,13.05
5,11,small_business,12.84
6,13,wedding,12.07
7,12,vacation,11.97
8,7,medical,11.65
9,4,home_improvement,11.59


**Null hypothesis (Ho): Interest rates does not vary with different purposes of Loans.**
    
**Alternate Hypothesis (Ha) : Interest rates varies for different purposes of loans.**
    
**Confidence Interval : 95% ; p-value : 0.05**

In [21]:
a1 = bankloans.loc[bankloans.Loan_Purpose == 'moving','Interest_Rate']
a2 = bankloans.loc[bankloans.Loan_Purpose == 'debt_consolidation','Interest_Rate']
a3 = bankloans.loc[bankloans.Loan_Purpose == 'house','Interest_Rate']
a4 = bankloans.loc[bankloans.Loan_Purpose == 'other','Interest_Rate']
a5 = bankloans.loc[bankloans.Loan_Purpose == 'credit_card','Interest_Rate']
a6 = bankloans.loc[bankloans.Loan_Purpose == 'small_business','Interest_Rate']
a7 = bankloans.loc[bankloans.Loan_Purpose == 'wedding','Interest_Rate']
a8 = bankloans.loc[bankloans.Loan_Purpose == 'vacation','Interest_Rate']
a9 = bankloans.loc[bankloans.Loan_Purpose == 'medical','Interest_Rate']
a10 = bankloans.loc[bankloans.Loan_Purpose == 'home_improvement','Interest_Rate']
a11 = bankloans.loc[bankloans.Loan_Purpose == 'car','Interest_Rate']
a12 = bankloans.loc[bankloans.Loan_Purpose == 'educational','Interest_Rate']
a13 = bankloans.loc[bankloans.Loan_Purpose == 'major_purchase','Interest_Rate']
a14 = bankloans.loc[bankloans.Loan_Purpose == 'renewable_energy','Interest_Rate']

In [22]:
relate_3 = stats.f_oneway(a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14)
relate_3

F_onewayResult(statistic=7.482212567168054, pvalue=1.1627206793499994e-14)

***p-value << 0.05, hence we can say we interest rates varies for different purposes of loans.***

# There is relationship between FICO scores and Home Ownership.

**Null hypothesis (Ho): There is no relationship between FICO score and the Home Ownership.**
    
**Alternate Hypothesis (Ha) : There is a relationship between FICO score and the Home Ownership.**
    
**Confidence Interval : 95% ; p-value : 0.05**

In [23]:
bankloans.Home_Ownership.nunique()

5

In [24]:
bankloans.FICO_Range.nunique()

38

In [25]:
observe = pd.crosstab(index = bankloans.Home_Ownership,columns = bankloans.FICO_Range)
observe

FICO_Range,640.0,645.0,650.0,655.0,660.0,665.0,670.0,675.0,680.0,685.0,...,780.0,785.0,790.0,795.0,800.0,805.0,810.0,815.0,820.0,830.0
Home_Ownership,,,,,,,,,,,,,,,,,,,,,
MORTGAGE,0,1,1,0,41,52,67,79,55,61,...,21,12,15,7,7,6,5,6,0,1
NONE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
OTHER,0,0,0,1,0,0,1,1,2,0,...,0,0,0,0,0,0,0,0,0,0
OWN,1,0,0,0,17,18,13,11,9,10,...,3,2,1,1,3,2,1,0,1,0
RENT,4,2,0,3,67,75,90,75,91,65,...,4,5,4,5,2,3,2,0,0,0


In [26]:
stats.chi2_contingency(observe)

(473.5700339166009,
 1.0046919010161217e-35,
 148,
 array([[2.29600e+00, 1.37760e+00, 4.59200e-01, 1.83680e+00, 5.74000e+01,
         6.65840e+01, 7.85232e+01, 7.62272e+01, 7.20944e+01, 6.24512e+01,
         6.42880e+01, 7.02576e+01, 6.10736e+01, 6.15328e+01, 5.14304e+01,
         4.27056e+01, 5.23488e+01, 4.31648e+01, 4.31648e+01, 2.98480e+01,
         2.43376e+01, 2.47968e+01, 2.80112e+01, 2.11232e+01, 2.11232e+01,
         1.65312e+01, 7.80640e+00, 1.01024e+01, 1.28576e+01, 8.72480e+00,
         9.18400e+00, 5.96960e+00, 5.96960e+00, 5.05120e+00, 3.67360e+00,
         2.75520e+00, 4.59200e-01, 4.59200e-01],
        [2.00000e-03, 1.20000e-03, 4.00000e-04, 1.60000e-03, 5.00000e-02,
         5.80000e-02, 6.84000e-02, 6.64000e-02, 6.28000e-02, 5.44000e-02,
         5.60000e-02, 6.12000e-02, 5.32000e-02, 5.36000e-02, 4.48000e-02,
         3.72000e-02, 4.56000e-02, 3.76000e-02, 3.76000e-02, 2.60000e-02,
         2.12000e-02, 2.16000e-02, 2.44000e-02, 1.84000e-02, 1.84000e-02,
         1.4

***p-value <<< 0.05  therefore we can say that there is relationship between FICO scores and Home Ownership. It means that, People 
with owning home will have high FICO scores.***